In [ ]:
import pandas as pd
import numpy as np
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

from atcs import ATCS
import fig_plot as fp


In [4]:
N_sample = 10
seed = 1
solver_type = 'minlp'
model_type = 'deterministic'
sample_type = f'head_{N_sample}'
data = ATCS(seed = 1)
data.choose_subset_point(N_sample, randomized = False) # Choose subset data

In [5]:
data.cc_df

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99
0,1,0,1,1,0,1,0,0,1,0,...,1,0,1,0,1,1,0,0,1,0
1,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,1
3,0,0,0,0,1,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,1,1,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,1,0,0,1,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,1
1068,0,0,1,1,0,1,0,0,1,0,...,0,0,1,0,0,1,0,1,1,1
1069,1,0,0,1,1,0,0,0,1,0,...,1,1,1,0,1,0,0,0,0,0
1070,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.r_s_sub_df

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99
index,,,,,,,,,,,,,,,,,,,,,
0,127.868274,99.607518,115.991472,98.862235,56.717849,93.655172,43.682917,39.445939,110.431507,49.762068,...,60.463201,78.619048,110.727024,41.199004,103.599169,85.335774,45.937307,40.648336,75.513443,68.891890
1,87.246488,78.262023,58.824388,58.741418,51.521360,49.331962,13.646325,70.820459,71.195141,93.745739,...,46.545122,95.152202,103.073344,110.604070,99.064217,78.637941,87.736412,117.169373,79.008553,82.904919
2,64.393799,64.664532,14.213737,47.586174,22.556134,81.748325,26.659697,22.664167,57.544840,97.072737,...,97.742631,34.963204,76.343352,65.430333,100.800704,113.249146,66.284062,58.608416,30.310834,109.036947
3,100.815352,19.522792,25.542728,105.072249,106.765349,91.871031,122.757547,46.847427,64.673271,64.345219,...,87.072585,90.600733,66.860000,101.362804,113.676205,58.621838,57.366586,67.822953,19.142513,35.151859
4,36.021544,79.813380,101.340367,112.704710,100.677891,42.712265,74.500984,95.005785,69.822481,62.174867,...,111.380887,111.434180,77.649138,46.212388,80.728188,128.209158,23.559988,56.324234,96.524725,11.287680
5,59.790297,75.464982,94.549237,107.297093,125.883371,122.332951,120.441396,67.282380,89.732463,66.399334,...,85.045689,95.888783,65.550104,118.632784,107.459446,54.973827,100.516791,134.978542,97.741017,123.414934
6,25.916223,52.835440,55.309130,106.616210,142.028568,22.591285,52.267996,58.075950,72.386911,75.414744,...,148.535597,127.382021,97.289303,102.641179,48.115249,86.870329,102.993912,81.643596,18.174664,164.248451
7,118.980835,128.342226,10.018831,50.889568,114.312505,126.315884,35.721376,45.712229,101.026755,91.089408,...,44.682210,116.616183,91.121869,42.267276,68.936708,97.235380,44.139266,90.498132,113.218165,68.177761
8,55.738322,52.479218,107.436029,72.132714,50.646508,78.868725,95.729358,33.997199,106.022438,81.116711,...,95.404962,127.680797,102.980897,75.200980,80.825857,13.315118,45.066952,53.783449,18.290175,78.272255
